In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import requests
from amadeus import Location
from amadeus import Client, ResponseError

In [2]:
amadeus = Client(
    client_id='mfWDFAuJHzCAyiuyrF2ZNPLt86x6pBHe',
    client_secret='20kJCr29aZVItX2F'
)

try:
    response = amadeus.reference_data.urls.checkin_links.get(airlineCode='BA')
    print(response.data)
except ResponseError as error:
    print(error)

[{'type': 'checkin-link', 'id': 'BAEN-GBAll', 'href': 'https://www.britishairways.com/travel/olcilandingpageauthreq/public/en_gb', 'channel': 'All'}, {'type': 'checkin-link', 'id': 'BAEN-GBMobile', 'href': 'https://www.britishairways.com/travel/managebooking/public/en_gb/device-mobile?bookingRef={PNR}&lastname={LAST}', 'channel': 'Mobile', 'parameters': {'LAST': {'description': 'Passenger Last Name', 'type': 'string'}, 'PNR': {'description': 'Record Locator', 'type': 'string', 'format': '([a-zA-Z]|[0-9]){6}'}}}, {'type': 'checkin-link', 'id': 'BAEN-GBWeb', 'href': 'https://www.britishairways.com/travel/managebooking/public/en_gb?&bookingRef={PNR}&lastname={LAST}', 'channel': 'Web', 'parameters': {'LAST': {'description': 'Passenger Last Name', 'type': 'string'}, 'PNR': {'description': 'Record Locator', 'type': 'string', 'format': '([a-zA-Z]|[0-9]){6}'}}}]


## Where did most people fly to from SFO in Valentineds day 2018? (from Feb 01 to Feb 28 - full month)

In [3]:
response_best_place = amadeus.travel.analytics.air_traffic.traveled.get(originCityCode='SFO', period='2018-02')
response_best_place = response_best_place.data
print(response_best_place)

[{'type': 'air-traffic', 'destination': 'LAS', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 63}, 'travelers': {'score': 100}}}, {'type': 'air-traffic', 'destination': 'LAX', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 86}, 'travelers': {'score': 93}}}, {'type': 'air-traffic', 'destination': 'NYC', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 100}, 'travelers': {'score': 91}}}, {'type': 'air-traffic', 'destination': 'CHI', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 52}, 'travelers': {'score': 36}}}, {'type': 'air-traffic', 'destination': 'BLR', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 17}, 'travelers': {'score': 13}}}, {'type': 'air-traffic', 'destination': 'MAD', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 27}, 'travelers': {'score': 11}}}, {'type': 'air-traffic', 'destination': 'SIN', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 14}, 'travelers': {'score': 10}}}, {'type': 'air-traffic', 

In [4]:
# print(json.dumps(response, indent=4, sort_keys=True))
print(json.dumps(response_best_place, indent=4, sort_keys=True))

[
    {
        "analytics": {
            "flights": {
                "score": 63
            },
            "travelers": {
                "score": 100
            }
        },
        "destination": "LAS",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 86
            },
            "travelers": {
                "score": 93
            }
        },
        "destination": "LAX",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 100
            },
            "travelers": {
                "score": 91
            }
        },
        "destination": "NYC",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 52
            },
            "travelers": {
                "score": 36
            }
     

# Store JSON data into a DataFrame #1

In [5]:
#json_file = "../Resources/customer_location.json"
best_places_df = pd.DataFrame.from_dict(json_normalize(response_best_place), orient='columns')
best_places_df

,analytics.flights.score,analytics.travelers.score,destination,subType,type
0,63,100,LAS,TRAVELED,air-traffic
1,86,93,LAX,TRAVELED,air-traffic
2,100,91,NYC,TRAVELED,air-traffic
3,52,36,CHI,TRAVELED,air-traffic
4,17,13,BLR,TRAVELED,air-traffic
5,27,11,MAD,TRAVELED,air-traffic
6,14,10,SIN,TRAVELED,air-traffic
7,21,7,BCN,TRAVELED,air-traffic
8,11,7,SNA,TRAVELED,air-traffic
9,17,5,IST,TRAVELED,air-traffic


# Clean DataFrame - #1 

## TRANSFORM: ADDING REGION OF AIRPORT TO THE DATA FRAME

In [6]:
# Get a specific city or airport based on its id

airport_codes = best_places_df['destination'].tolist()
# airport_codes

test_list = []

for airport in airport_codes:
    try:
        response = amadeus.reference_data.locations.get(
        keyword=airport,
        subType=Location.ANY)

     #  print(airport) 

        test_list.append(response.data[0]['address']['regionCode'])

    except:
        test_list.append('not here')
    

best_places_df['country'] = pd.DataFrame(test_list)

In [7]:
best_places_df

,analytics.flights.score,analytics.travelers.score,destination,subType,type,country
0,63,100,LAS,TRAVELED,air-traffic,NAMER
1,86,93,LAX,TRAVELED,air-traffic,NAMER
2,100,91,NYC,TRAVELED,air-traffic,NAMER
3,52,36,CHI,TRAVELED,air-traffic,NAMER
4,17,13,BLR,TRAVELED,air-traffic,ASIA
5,27,11,MAD,TRAVELED,air-traffic,EUROP
6,14,10,SIN,TRAVELED,air-traffic,ASIA
7,21,7,BCN,TRAVELED,air-traffic,EUROP
8,11,7,SNA,TRAVELED,air-traffic,NAMER
9,17,5,IST,TRAVELED,air-traffic,not here


In [8]:
# print(json.dumps(response.data, indent=4, sort_keys=True))
# print(response.data[0]['address']['countryName'])

# TRANSFORM: GET RID OF ROWS WE DONT NEED FROM THE JSON CALL

In [9]:
best_places_df_new = best_places_df[best_places_df.country != 'NAMER']
best_places_df_new = best_places_df_new[best_places_df.country != 'not here']
best_places_df_new

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,analytics.flights.score,analytics.travelers.score,destination,subType,type,country
4,17,13,BLR,TRAVELED,air-traffic,ASIA
5,27,11,MAD,TRAVELED,air-traffic,EUROP
6,14,10,SIN,TRAVELED,air-traffic,ASIA
7,21,7,BCN,TRAVELED,air-traffic,EUROP


# Take the top 3 destination airports and we query the price of the flights for same date range in 2019


In [10]:
#testing the api call
low_price = amadeus.shopping.flight_offers.get(origin='SFO', destination='JFK', departureDate='2020-02-14')
low_price.data

[{'type': 'flight-offer',
  'id': '1565313772516-586128751',
  'offerItems': [{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'SFO',
          'terminal': '1',
          'at': '2020-02-14T18:50:00-08:00'},
         'arrival': {'iataCode': 'SLC',
          'terminal': '2',
          'at': '2020-02-14T21:44:00-07:00'},
         'carrierCode': 'DL',
         'number': '381',
         'aircraft': {'code': '319'},
         'operating': {'carrierCode': 'DL', 'number': '381'},
         'duration': '0DT1H54M'},
        'pricingDetailPerAdult': {'travelClass': 'ECONOMY',
         'fareClass': 'H',
         'availability': 9,
         'fareBasis': 'HA7NA0MQ'}},
       {'flightSegment': {'departure': {'iataCode': 'SLC',
          'terminal': '2',
          'at': '2020-02-14T23:40:00-07:00'},
         'arrival': {'iataCode': 'JFK',
          'terminal': '4',
          'at': '2020-02-15T06:16:00-05:00'},
         'carrierCode': 'DL',
         'number': '1202',
         'air

In [11]:
# low_price_df = pd.DataFrame.from_dict(json_normalize(low_price.data), orient='columns')
# low_price_df

# Defining the json objects we will append to the table. Price, Departure_date and Arrival_date
    

In [12]:
# price == low_price.data[0]['offerItems'][0]['price']['total']
# departure_date == low_price.data[0]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at']
# arrival_date == low_price.data[0]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at']

In [13]:
# Creating a list of final airport codes that we will loop through for API call
destinations = best_places_df_new['destination'].tolist()
destinations

['BLR', 'MAD', 'SIN', 'BCN']

In [ ]:
# low_price_list = []
# low_price_df = {}

# for destination in destinations:
    
#     response = amadeus.shopping.flight_offers.get(origin='SFO', destination=destination, departureDate='2020-02-14')
#     price_list = []
#     departure_list = []
#     arrival_list = []
    
#     for i in range(len(low_price.data)):
#         try:
#             low_price_list.append(destination)
#             low_price_df['destination'] = destination
          
#             price_list.append(response.data[i]['offerItems'][0]['price']['total'])
#             low_price_df['price'] = price_list
  
#             departure_list.append(response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'])
#             low_price_df['departure_date'] = departure_list  
                    
#             arrival_list.append(response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at'])
#             low_price_df['arrival_date'] = arrival_list  
            
#         except:
#             low_price_list.append('not here')

In [29]:
results_leave = []
for destination in destinations:
    
    response = amadeus.shopping.flight_offers.get(origin='SFO', destination=destination, departureDate='2020-02-14')
#     price_list = []
#     departure_list = []
#     arrival_list = []
    
    for i in range(len(low_price.data)):
        try:
            item = {'destination': destination, \
                    'price_leave': response.data[i]['offerItems'][0]['price']['total'], \
                    'departure_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'], \
                    'arrival_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at']
                   }
            results_leave.append(item)
        
            
        except:
            pass

In [31]:
# to pandas DF
results_df = pd.DataFrame(results_leave)
results_df.head()

,arrival_date,departure_date,destination,price_leave
0,2020-02-14T14:52:00-05:00,2020-02-14T06:15:00-08:00,BLR,1085.88
1,2020-02-14T14:52:00-05:00,2020-02-14T06:15:00-08:00,BLR,919.19
2,2020-02-14T19:19:00-08:00,2020-02-14T16:55:00-08:00,BLR,1036.18
3,2020-02-15T10:35:00Z,2020-02-14T16:10:00-08:00,BLR,4197.60
4,2020-02-14T17:07:00-05:00,2020-02-14T08:30:00-08:00,BLR,973.19


query the low price flight to SFO (Feb 16,2020)

In [18]:
results_back = []
for destination in destinations:
    
    response = amadeus.shopping.flight_offers.get(origin=destination, destination='SFO', departureDate='2020-02-16')
    
    for i in range(len(low_price.data)):
        try:
            item = {'origin': destination, \
                    'price': response.data[i]['offerItems'][0]['price']['total'], \
                    'departure_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['departure']['at'], \
                    'arrival_date': response.data[i]['offerItems'][0]['services'][0]['segments'][0]['flightSegment']['arrival']['at']
                   }
            results_back.append(item)
        
            
        except:
            pass

In [28]:
# to pandas DF
results_df_back = pd.DataFrame(results_back)
results_back

[{'origin': 'BLR',
  'price': '1107.82',
  'departure_date': '2020-02-16T01:55:00+05:30',
  'arrival_date': '2020-02-16T08:10:00+01:00'},
 {'origin': 'BLR',
  'price': '983.99',
  'departure_date': '2020-02-16T03:35:00+05:30',
  'arrival_date': '2020-02-16T09:00:00+01:00'},
 {'origin': 'BLR',
  'price': '927.47',
  'departure_date': '2020-02-16T03:35:00+05:30',
  'arrival_date': '2020-02-16T09:00:00+01:00'},
 {'origin': 'BLR',
  'price': '1112.89',
  'departure_date': '2020-02-16T01:55:00+05:30',
  'arrival_date': '2020-02-16T08:10:00+01:00'},
 {'origin': 'BLR',
  'price': '932.54',
  'departure_date': '2020-02-16T03:35:00+05:30',
  'arrival_date': '2020-02-16T09:00:00+01:00'},
 {'origin': 'BLR',
  'price': '1280.89',
  'departure_date': '2020-02-16T01:55:00+05:30',
  'arrival_date': '2020-02-16T08:10:00+01:00'},
 {'origin': 'BLR',
  'price': '982.59',
  'departure_date': '2020-02-16T03:35:00+05:30',
  'arrival_date': '2020-02-16T09:00:00+01:00'},
 {'origin': 'BLR',
  'price': '1086.02

find cheapest flight back from each destination

In [27]:
back_lowest = results_df_back.groupby('origin').min()['price']
back_lowest = pd.DataFrame(back_lowest)
back_lowest

,price
origin,
BCN,2264.79
BLR,1086.02
MAD,2111.39
SIN,1082.02


find cheapest flight to each destination

# Connect to local database

In [16]:
rds_connection_string = "postgres:Afs200708-@localhost:5432/"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [17]:
# engine.table_names()